In [2]:
#Load libraries 
import pandas as pd
import sspa
import scipy
import numpy as np 
import os #get path location
import pickle

We make two different networks, one for the COVID cases 1-2 compared to COVID cases 3-7 <br>
This is because there are only 18 samples in common between the metabolomic and proteomic datasets

0       Common samples: 18           Metabolomic samples: 133        Proteomic samples: 123 <br>
1-2       Common samples: 45          Metabolomic samples: 45        Proteomic samples: 48 <br>
3-4       Common samples: 56          Metabolomic samples: 57        Proteomic samples: 59 <br>
5-7       Common samples: 27          Metabolomic samples: 28        Proteomic samples: 28 <br>

146 common samples overall,   128 cases (45 samples (WHO 1-2) vs 83 samples (WHO 3-7))

In [3]:
df = pd.read_csv('Data/Su_COVID_metabolomics_processed_commoncases.csv', index_col=0)
df_mild = (df[df["WHO_status"] == '1-2']).iloc[:,:-2] #45 samples, remove the metadata
df_severe = (df[(df["WHO_status"] == '3-4') | (df["WHO_status"] == '5-7')]).iloc[:,:-2] #83 samples

Check that data is centred at zero (mean=zero and standard deviation=1 for each analyte):

In [4]:
df_num  = df.iloc[:,:-2] #all rows, all columns apart from last two
print(df_num.max().max())
print(df_num.min().min())
print(df_num.mean(axis = 0)) #mean of 0
print(df_num.std(axis = 0)) #sd of 1

10.947848832735254
-5.091869731580215
1372    -6.938894e-18
16610   -4.163336e-17
72665    4.163336e-17
27823    0.000000e+00
30915   -4.163336e-17
             ...     
28238   -3.816392e-17
76341    3.122502e-17
89312    2.428613e-17
17861    2.081668e-17
89188    6.938894e-17
Length: 333, dtype: float64
1372     1.003929
16610    1.003929
72665    1.003929
27823    1.003929
30915    1.003929
           ...   
28238    1.003929
76341    1.003929
89312    1.003929
17861    1.003929
89188    1.003929
Length: 333, dtype: float64


In [9]:
df

,1372,16610,72665,27823,30915,37373,16831,545959,17050,16359,...,133693,133694,28036,28238,76341,89312,17861,89188,WHO_status,Group
sample_id,,,,,,,,,,,,,,,,,,,,,
INCOV001,-0.342522,-0.830841,-0.703451,-0.451278,0.153207,-0.443252,-0.228725,0.501998,-0.861886,-0.344155,...,-0.465505,0.055753,-0.225622,0.400398,0.163210,1.579262,-0.162221,-0.603790,3-4,COVID19
INCOV002,-0.125585,-1.002246,0.399273,-0.828341,-0.980576,0.373921,-1.076979,-0.131876,-0.869093,-0.467326,...,0.509336,0.627269,0.321718,-0.158514,0.001469,-1.051485,1.299330,0.578560,5-7,COVID19
INCOV003,-0.859326,-0.409159,3.573100,-0.546783,-0.993647,1.547649,0.370131,-0.020399,-0.703786,-0.015627,...,2.056610,1.717149,-0.297723,-1.276891,-0.658288,2.141594,-1.091183,0.447197,5-7,COVID19
INCOV004,-0.562999,-1.560249,-1.269383,-1.437919,0.108734,-0.192066,0.161591,-0.586759,-1.457246,-0.500612,...,0.793065,-0.033125,-0.366082,-0.642671,1.724436,2.074193,-1.021184,-0.410849,3-4,COVID19
INCOV005,-0.741957,0.387909,-0.711036,-0.711515,0.670720,0.610591,-0.322241,-0.793451,-1.230681,-0.246493,...,-0.184967,-0.469553,-0.293869,0.558366,-0.390308,-0.455737,-0.535223,-0.847727,3-4,COVID19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INCOV135,-2.075777,-0.485936,-0.671225,0.069483,0.042579,0.357250,-0.505272,-0.378441,-0.444915,-0.368108,...,-0.516557,-0.392716,-0.328170,1.013774,-0.391566,-0.407585,1.041224,-0.866190,1-2,COVID19
INCOV136,-0.174602,1.167644,0.055499,0.113342,-0.663851,0.799367,0.004697,0.748894,1.328753,0.391221,...,-0.741578,-0.730981,-0.380517,-1.251641,0.389163,-0.603048,0.941619,-0.511694,3-4,COVID19
INCOV137,0.274900,-0.014841,-0.928512,0.885263,-1.591862,0.230250,0.057818,-0.775962,-0.179700,-0.534786,...,-1.306132,-1.246428,1.763824,1.258387,0.448359,-0.508931,0.268474,-0.997206,5-7,COVID19


In [6]:
#Download the reactome pathways
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

#Download the root pathways
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}
root_pathway_names = list(root_pathway_dict.keys())



### Step 1: Determine initial test-statistic

In [7]:
#Function to calculate the squared Spearman correlation matrix 

def squared_spearman_corr(data):
    kpca_scores = sspa.sspa_kpca(data, reactome_pathways)   
    kpca_scores = kpca_scores.drop(columns = list(set(root_pathway_names) & set(kpca_scores.columns))) #using Sara's code to drop root pathways

    spearman_results = scipy.stats.spearmanr(kpca_scores)
    squared_spearman_coef = np.square(spearman_results[0]) #correlation coefficients (spearman_results[1] gives the p-values)

    return squared_spearman_coef,list(kpca_scores.columns)




#Function to calculate the absolute difference between two matrices and then determine the mean for each edge

def absolute_val(data1,data2,edgelist):
    abs_rho_squared = np.absolute(np.array(data1) - np.array(data2))

    #Mask the upper half of the dataframe (so I don't view the comparisons between the two same genes, and also the duplicate comparisons are removed)
    mask = abs_rho_squared.copy()
    mask = np.triu(np.ones(mask.shape)).astype(bool)
    mask = np.invert(mask) #invert true and false values so the diagonal is False as well
    non_dup_abs_squared = pd.DataFrame(abs_rho_squared, columns = edgelist, index = edgelist)
    non_dup_abs_squared = pd.DataFrame(non_dup_abs_squared).where(mask) #Replace all false values with NaN using mask

    abs_squared_list = non_dup_abs_squared.stack().reset_index()
    abs_squared_list['level_0'] = abs_squared_list["level_0"].astype(str) + ", " + abs_squared_list['level_1']
    abs_squared_list.columns = ["Edges","na","Initial_tstat"]
    abs_squared_list.index = abs_squared_list["Edges"]
    abs_squared_list = abs_squared_list.drop(columns = ["Edges","na"])

    return(abs_squared_list)

Note: For the delta squared correlation values for the unshuffled data (i.e. the real data) I keep the indices (pathway edges). Since I already have a record of the edges, there is no need to keep the edges for each permutation, since the order is the same each time. 

In [8]:
spearman_mild,edgelist = squared_spearman_corr(df_mild)
spearman_severe,edgelist = squared_spearman_corr(df_severe)

output = absolute_val(spearman_mild,spearman_severe,edgelist)

Note: There are no values with a Spearman correlation value of zero. The reason why some initial test statistics have a value of zero is because for both groups both values are zero.

In [13]:
test_df = pd.DataFrame(spearman_mild, columns = edgelist, index = edgelist)
display(test_df.iloc[:25,:25])

,R-HSA-110331,R-HSA-112310,R-HSA-112311,R-HSA-112315,R-HSA-1237112,R-HSA-1368082,R-HSA-1368108,R-HSA-1428517,R-HSA-1483148,R-HSA-1483206,...,R-HSA-15869,R-HSA-1592230,R-HSA-159418,R-HSA-1614603,R-HSA-1614635,R-HSA-163685,R-HSA-1655829,R-HSA-1660661,R-HSA-168249,R-HSA-189200
R-HSA-110331,1.000000,0.007222,0.006043,0.020738,0.378411,0.021004,0.021004,7.495620e-02,5.624209e-04,0.001155,...,0.785290,0.021004,0.056305,0.013843,0.049168,0.019726,0.021004,0.037154,0.001722,0.029562
R-HSA-112310,0.007222,1.000000,0.063791,0.809052,0.018884,0.715462,0.715462,7.655173e-06,3.585785e-01,0.357160,...,0.009974,0.715462,0.022441,0.119794,0.000202,0.516732,0.715462,0.016333,0.006651,0.015240
R-HSA-112311,0.006043,0.063791,1.000000,0.075607,0.061418,0.004016,0.004016,9.764252e-05,2.299678e-02,0.045219,...,0.042190,0.004016,0.006673,0.066617,0.215448,0.063592,0.004016,0.017848,0.003225,0.002736
R-HSA-112315,0.020738,0.809052,0.075607,1.000000,0.000157,0.522811,0.522811,2.078111e-03,2.709758e-01,0.288252,...,0.019504,0.522811,0.005463,0.086943,0.000037,0.333776,0.522811,0.016031,0.001744,0.022048
R-HSA-1237112,0.378411,0.018884,0.061418,0.000157,1.000000,0.000230,0.000230,1.855913e-02,1.488464e-02,0.026006,...,0.214837,0.000230,0.018416,0.000202,0.239571,0.139122,0.000230,0.002667,0.003150,0.017883
R-HSA-1368082,0.021004,0.715462,0.004016,0.522811,0.000230,1.000000,1.000000,1.924886e-03,2.767675e-01,0.269332,...,0.039580,1.000000,0.062467,0.060637,0.003999,0.371152,1.000000,0.004340,0.000674,0.000177
R-HSA-1368108,0.021004,0.715462,0.004016,0.522811,0.000230,1.000000,1.000000,1.924886e-03,2.767675e-01,0.269332,...,0.039580,1.000000,0.062467,0.060637,0.003999,0.371152,1.000000,0.004340,0.000674,0.000177
R-HSA-1428517,0.074956,0.000008,0.000098,0.002078,0.018559,0.001925,0.001925,1.000000e+00,6.249121e-07,0.000934,...,0.095293,0.001925,0.014981,0.270016,0.015240,0.002389,0.001925,0.020813,0.007022,0.008751
R-HSA-1483148,0.000562,0.358578,0.022997,0.270976,0.014885,0.276767,0.276767,6.249121e-07,1.000000e+00,0.866690,...,0.000744,0.276767,0.000108,0.005600,0.026177,0.123841,0.276767,0.000045,0.007088,0.010670
R-HSA-1483206,0.001155,0.357160,0.045219,0.288252,0.026006,0.269332,0.269332,9.343131e-04,8.666899e-01,1.000000,...,0.010000,0.269332,0.003106,0.009074,0.039109,0.152707,0.269332,0.010642,0.000071,0.003721


In [9]:
output

,Initial_tstat
Edges,
"R-HSA-112310, R-HSA-110331",0.007044
"R-HSA-112311, R-HSA-110331",0.023605
"R-HSA-112311, R-HSA-112310",0.285484
"R-HSA-112315, R-HSA-110331",0.008600
"R-HSA-112315, R-HSA-112310",0.057525
...,...
"R-HSA-975634, R-HSA-9717207",0.035746
"R-HSA-975634, R-HSA-9734207",0.016982
"R-HSA-975634, R-HSA-9735804",0.000969


In [10]:
output.to_csv("Data/permutation_test_files_metabolomics/initial_tstats.csv")

### Step 2: Shuffle the labels

The sample labels are shuffled, rather than assigning the samples to two different groups (since the sizes of the 1-2 class with the 3-7 class is not equal). See HPC_permutation_script_metabolomic for more info.

### Step 3: Read in the permutation files 

Using the HPC, 10 files each store 10k permutations. 10 array jobs are carried out to read in all 10k values, make them absolute and count how many are above the initial test statistic. See permutation_distribution.ipynb for more info.

https://www.jwilber.me/permutationtest/# <br>
We take the absolute value for the absolute difference between the two groups to compute the initial test statistic. Each permutation calculates a new test statistic calculated from the ABSOLUTE difference between the two test groups. It makes no difference whether I take the absolute difference or regular difference between the two, since either way it's a two-tailed p-value distribution, just that with the absolute difference the negative values have been mapped to the positive side.

### Step 4: Compare the difference in edges with other networks

Comparing with the naive networks (mild vs severe) to see if they agree with the differential network results:

In [4]:
import networkx as nx

In [5]:
mild = nx.read_gml("Cytoscape/metabolomic_final_mildcases.gml")
severe = nx.read_gml("Cytoscape/metabolomic_final_severecases.gml")

In [14]:
mild_edges = list(mild.edges())
print(len(mild_edges))
severe_edges = list(severe.edges())
print(len(severe_edges))

144
162


In [16]:
#Example code from https://stackoverflow.com/questions/41125909/python-find-elements-in-one-list-that-are-not-in-the-other
list_1=["a", "b", "c", "d", "e"]
list_2=["a", "f", "c", "m"]
set(list_2) - set(list_1)

#set(['m', 'f'])

{'f', 'm'}

In [12]:
#Edges present in the mild network but not the severe network
len(set(mild_edges) - set(severe_edges))

98

In [13]:
#Edges present in the severe network but not the mild network
len(set(severe_edges) - set(mild_edges))

116

In [1]:
#Reading in the edges expressed in the differential metabolomic network
with open('Data/permutation_test_files_metabolomics/sig_edges_beforefdr.txt') as f:
    lines = f.readlines()

edges_remaining = []

edges = lines[0].split(",")

for index in range(0,len(edges),2):
    list1 = edges[index],(edges[index+1][1:])
    edges_remaining.append(tuple(list1))

print(len(edges_remaining))
edges_remaining

795


[('R-HSA-1368082', 'R-HSA-1237112'),
 ('R-HSA-1368108', 'R-HSA-1237112'),
 ('R-HSA-1483206', 'R-HSA-1368082'),
 ('R-HSA-1483206', 'R-HSA-1368108'),
 ('R-HSA-1483255', 'R-HSA-1483206'),
 ('R-HSA-1483257', 'R-HSA-112310'),
 ('R-HSA-1483257', 'R-HSA-1368082'),
 ('R-HSA-1483257', 'R-HSA-1368108'),
 ('R-HSA-1483257', 'R-HSA-1483148'),
 ('R-HSA-156580', 'R-HSA-110331'),
 ('R-HSA-156582', 'R-HSA-1237112'),
 ('R-HSA-156582', 'R-HSA-1483206'),
 ('R-HSA-156582', 'R-HSA-1483257'),
 ('R-HSA-156584', 'R-HSA-156580'),
 ('R-HSA-15869', 'R-HSA-156584'),
 ('R-HSA-1592230', 'R-HSA-1237112'),
 ('R-HSA-1592230', 'R-HSA-1483206'),
 ('R-HSA-1592230', 'R-HSA-1483257'),
 ('R-HSA-1614603', 'R-HSA-1428517'),
 ('R-HSA-1655829', 'R-HSA-1237112'),
 ('R-HSA-1655829', 'R-HSA-1483206'),
 ('R-HSA-1655829', 'R-HSA-1483257'),
 ('R-HSA-168249', 'R-HSA-156582'),
 ('R-HSA-192456', 'R-HSA-1428517'),
 ('R-HSA-192456', 'R-HSA-1483206'),
 ('R-HSA-192456', 'R-HSA-1483257'),
 ('R-HSA-196071', 'R-HSA-110331'),
 ('R-HSA-196071', '

In [21]:
intersection = list(set(edges_remaining).intersection(list(set(mild_edges))))  
len(intersection) #30 edges out of 98

30

In [22]:
intersection = list(set(edges_remaining).intersection(list(set(severe_edges)))) 
len(intersection) #6 edges out of 116

6